# ДЗ Пример влияния лишних признаков на качество модели

In [45]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings 
warnings.filterwarnings("ignore") 

from sklearn.linear_model import LinearRegression

Генерируем датасет и признаки. Задаем значение целевой переменной через функцию:  

**продажи магазина(revenue) в день  = средний чек (average order value -aov) * количество покупателей (traffic)**

в датесете будет еще один дополнительный признак

cost -  расходы



In [46]:

n_samples = 1000

aov = np.random.choice(200, n_samples) + 100     
traffic = np.random.choice(1000, n_samples) + 1
cost = np.random.choice(500, n_samples) + 10

revenue = aov * traffic  + 300  #выручка магазина   

data = pd.DataFrame({'cost': cost, 'traffic': traffic, 'aov': aov, 'revenue': revenue})
data.head(5)

,cost,traffic,aov,revenue
0,142,178,184,33052
1,504,604,294,177876
2,193,956,217,207752
3,296,220,148,32860
4,407,813,176,143388


строим модель линейной регресси по созданному датасету

In [47]:
from sklearn.metrics import mean_absolute_error

X = data[['cost', 'traffic', 'aov']]
y = data['revenue']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['cost', 'traffic', 'aov']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ -1.25368193 197.1257485  512.92736005]
Bias: -101090.05845893813
Error: 12355.562246908568


по модели получили вес для каждого из признаков 
  
+ cost -1.25368193    - вес незначительный, но тем не менее влияет на предсказанное значение

+ traffic  197.1257485 

+ revenue  512.92736005

In [48]:
y.median()

92864.5

постром еще одну модель, но используем только значимые признаки:

In [49]:
X = data[['traffic', 'aov']]
y = data['revenue']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['traffic', 'aov']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [197.14062877 512.81348869]
Bias: -101395.91836649382
Error: 12354.930906074345


по модели получили вес для каждого из признаков 

+ traffic  197.14062877 

+ revenue  512.81348869

Веса и ошибка почти не изменились 

Рассчитаем вручную признак "продажи (revenue_m)" и построим модель только от этого признакак

In [50]:
# Создаем новый признак
data['revenue_m'] = data['traffic'] * data['aov']
data.head(5)

,cost,traffic,aov,revenue,revenue_m
0,142,178,184,33052,32752
1,504,604,294,177876,177576
2,193,956,217,207752,207452
3,296,220,148,32860,32560
4,407,813,176,143388,143088


In [51]:
X = data[['revenue_m']]
y = data['revenue']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['revenue_m']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [1.]
Bias: 300.0000000000291
Error: 1.8416528746456608e-11


По одному признаку модель правильно определила сдвиг в исходно заданной функции:

Вес единственного признака:
Weights: [1.]

Сдвиг равен заданному значению 300:
Bias: 300.00000000001455

Ошибка близка к 0
Error: 4.566231837088708e-12


In [52]:
y.median()

92864.5